In [ ]:
import altair as alt
import pandas as pd
import json


In [ ]:

# Load the GeoJSON file for NYC map
with open('Borough Boundaries.geojson') as f:
    nyc_geojson = json.load(f)

# Load the dataset
data = pd.read_csv('2022_Motor_Vehicle_Collisions_-_Crashes.csv')

# Convert borough names in data to match the format in the GeoJSON file (title case)
data['BOROUGH'] = data['BOROUGH'].str.title()

# Aggregate data by borough
borough_data = data.groupby('BOROUGH').size().reset_index(name='Incident Count')

# Convert GeoJSON into a format Altair can use
nyc_borough_geojson_altair = alt.InlineData(values=nyc_geojson, format=alt.DataFormat(property='features', type='json'))

# Create the choropleth chart
choropleth = alt.Chart(nyc_borough_geojson_altair).mark_geoshape(
    stroke='black',
    strokeWidth=1
).encode(
    color='Incident Count:Q',
    tooltip=['properties.boro_name:N', 'Incident Count:Q']
).transform_lookup(
    lookup='properties.boro_name',
    from_=alt.LookupData(borough_data, 'BOROUGH', ['Incident Count'])
).properties(
    title='NYC Motor Vehicle Collisions by Borough',
    width=500,
    height=300
)

choropleth